Example 1 : **y=x^2**

without using autograd operation

In [1]:
def dy_dx(x):
  return 2*x

In [2]:
dy_dx(3)

6

Here's How we can calculate gradient without defining manually function
- first while creating tensor x , write requires_grad=True
- after that calculated y in forward direction, which will remember the mathematical operation going to perform
for exp: here is power operation
- then simply did y.backward() , during nackward direction all gradinets will get calculated
- to see gradient values , simply write x.grad

with using autograd operations

In [3]:
import torch

In [4]:
x = torch.tensor(3.0, requires_grad=True)

In [5]:
y = x**2

In [6]:
x

tensor(3., requires_grad=True)

In [7]:
y

tensor(9., grad_fn=<PowBackward0>)

In [8]:
y.backward()

In [9]:
x.grad

tensor(6.)

Eample2 : Calculate differentiation of nested function
**y=x^2** , **z=sin(y)**

In [10]:
# Without using autograd operations


import math

def dz_dx(x):
    return 2 * x * math.cos(x**2)

In [11]:
dz_dx(4)

-7.661275842587077

In [12]:
# With using autograd operations
x = torch.tensor(4.0, requires_grad=True)

In [13]:
y = x ** 2

In [14]:
z = torch.sin(y)

In [15]:
x

tensor(4., requires_grad=True)

In [16]:
y

tensor(16., grad_fn=<PowBackward0>)

In [17]:
z

tensor(-0.2879, grad_fn=<SinBackward0>)

In [18]:
z.backward()

In [19]:
x.grad

tensor(-7.6613)

In [20]:
y.grad

<ipython-input-20-10b3a7061f6d>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  y.grad


yaha pe dayan dena , jo humara gradients calculate ho rha hai voh leaf tensors ke upar hota h , intermediate tensor ke liye calculation nhi hota h , but agr if you want to generate derivative of intermediate tensors then we can , we'll see it later

# Lets calculate gradient of small data where cgpa(6.7) input | placement(0) output

**Calculating manually each step for gradient**

In [23]:
import torch

# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [24]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [25]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [26]:
loss

tensor(6.7012)

In [27]:
# Derivatives ---Backpropagation:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [28]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


**Calculating gradient easily using Autograd**

In [30]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

In [31]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [32]:
w

tensor(1., requires_grad=True)

In [33]:
b

tensor(0., requires_grad=True)

In [34]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [35]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [36]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [37]:
loss.backward()

In [38]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


we can also create vector tensors [,,,] , as before we have created some scaler tensor  (x, y , w , b)

In [39]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

In [40]:
x

tensor([1., 2., 3.], requires_grad=True)

In [41]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [42]:
y.backward()

In [43]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

after every epoch (forward and backward pass) , gradients value need to be clear otherwise our gradient value will gets accumulated (adding previous gradient with current gradient), which is not good and we'll never gets correct gradient value , thats why we need to clear grad after every epoch using **x.grad.zero_()**

In [44]:
# clearing grad
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [45]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [46]:
y.backward()

In [47]:
x.grad

tensor(4.)

In [48]:
x.grad.zero_()

tensor(0.)

In some situations sometime we dont need to calculate gradient , so we can disable gradient tracking

for ex: after model training , we can stop calculating gradient during model prediction we dont need gradient

In [49]:
# disable gradient tracking
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [50]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [51]:
y.backward()

In [52]:
x.grad

tensor(4.)

To make sure that only forward tracking will work and calculating gradint(backward pass tracking gets stop) we have 3 options for that

In [57]:
# option 1 - requires_grad_(False)
# option 2 - detach()
# option 3 - torch.no_grad()

In [56]:
x.requires_grad_(False) #_ underscore se inplace changes hoge

tensor(2.)

In [58]:
x

tensor(2.)

In [59]:
y = x ** 2

In [60]:
y

tensor(4.)

In [61]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [62]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [64]:
z = x.detach()  # in detach we will build a completely new tensor where gradient will not get calculated kuki ye z computional graph se alg ho chuka hai aur yaha gradient tracking on nhi n
z

tensor(2.)

In [65]:
y = x ** 2

In [66]:
y

tensor(4., grad_fn=<PowBackward0>)

In [67]:
y1 = z ** 2
y1

tensor(4.)

In [68]:
y.backward()

In [69]:
y1.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [71]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [72]:
# option 3: by using no_grad funtion
with torch.no_grad():
    y = x ** 2

In [73]:
y

tensor(4.)

In [74]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn